In [20]:
import pandas
import numpy

In [134]:
raw_df=pandas.read_csv('pitchdata.csv')
raw_df.head()

,GameId,PitcherId,HitterId,PitcherSide,HitterSide,PrimaryEvent,PitcherTeamId,HitterTeamId,PA,AB,H,2B,3B,HR,TB,BB,SF,HBP
0,2016/09/15/clemlb-chamlb-1,448306,424825,R,L,GO,145,114,1,1,0,0,0,0,0,0,0,0
1,2016/09/15/clemlb-chamlb-1,448306,543401,R,L,BHA,145,114,0,0,0,0,0,0,0,0,0,0
2,2016/09/15/clemlb-chamlb-1,448306,543401,R,L,GO,145,114,1,1,0,0,0,0,0,0,0,0
3,2016/09/15/clemlb-chamlb-1,448306,608070,R,L,FO,145,114,1,1,0,0,0,0,0,0,0,0
4,2016/09/15/clemlb-chamlb-1,605182,594809,R,L,KS,114,145,1,1,0,0,0,0,0,0,0,0


In [7]:
raw_df.shape

(20066, 18)

In [152]:
def create_combo_dataframe(raw_df, stat, subject, split):
    
    split_char=split[3] #get L or R split character
    
    #
    if split[5]=='H':
        
        split_filter='HitterSide'
        
    elif split[5]=='P':
        
        split_filter='PitcherSide'
        
    else:
        
        raise Exception('Unknown split type or data error')
        

    combo_df=raw_df[raw_df[split_filter]==split_char]

    combo_df=combo_df.groupby(subject).agg({'PA': numpy.sum, 'AB': numpy.sum,'H': numpy.sum,'2B': numpy.sum,'3B': numpy.sum,'HR': numpy.sum,'TB': numpy.sum,'BB': numpy.sum,'SF': numpy.sum,'HBP': numpy.sum,})

    combo_df=combo_df[combo_df['PA']>=25]
    
    combo_df['SubjectId']=combo_df.index
    
    combo_df['Stat']=stat
    
    combo_df['Split']=split
    
    combo_df['Subject']=subject

    if stat=='AVG':
        
        combo_df['Value']=(combo_df['H']
                           / combo_df['AB'])
    
    elif stat=='OBP':
        
        combo_df['Value']=((combo_df['H']
                           + combo_df['BB']
                           + combo_df['HBP'])
                           / (combo_df['AB']
                             + combo_df['BB']
                             + combo_df['HBP']
                             + combo_df['SF']))
    
    elif stat=='SLG':
        
        combo_df['Value']=(combo_df['TB']
                           / combo_df['AB'])
        
    elif stat=='OPS':
        
        combo_df['Value']=(((combo_df['H']
                           + combo_df['BB']
                           + combo_df['HBP'])
                           / (combo_df['AB']
                             + combo_df['BB']
                             + combo_df['HBP']
                             + combo_df['SF']))
                               + (combo_df['TB']
                               / combo_df['AB']))

    else:
        
        raise Exception('Unknown or malformed stat request')
        
    combo_df['Value']=combo_df['Value'].round(3)
    
    return combo_df[['SubjectId', 'Stat', 'Split', 'Subject' , 'Value']]

In [149]:
#AVG,HitterId,vs RHP
'''SELECT 
  HitterId AS SubjectId,
  'AVG' AS Stat,
  'vs RHP' AS Split,
  'HitterId' AS Subject,
  ROUND(CAST(SUM(H) AS FLOAT)/SUM(AB), 3) AS Value
FROM ./data/raw/pitchdata.csv
WHERE PitcherSide = 'R'
GROUP BY HitterId
HAVING SUM(PA) >= 25
'''

#output_df_headers = SubjectID, Stat, Split, Subject, Value (which is the calculated stat)

testdf1=create_combo_dataframe(raw_df, 'AVG', 'HitterId', 'vs RHP')

testdf2=create_combo_dataframe(raw_df, 'AVG', 'HitterId', 'vs LHP')

testdf1=testdf1.append(testdf2)

testdf1



HBP            int64
3B             int64
SF             int64
AB             int64
H              int64
TB             int64
PA             int64
BB             int64
HR             int64
2B             int64
SubjectId      int64
Stat          object
Split         object
Subject       object
Value        float64
dtype: object
HBP            int64
3B             int64
SF             int64
AB             int64
H              int64
TB             int64
PA             int64
BB             int64
HR             int64
2B             int64
SubjectId      int64
Stat          object
Split         object
Subject       object
Value        float64
dtype: object


,SubjectId,Stat,Split,Subject,Value
HitterId,,,,,
120074,120074,AVG,vs RHP,HitterId,0.277
134181,134181,AVG,vs RHP,HitterId,0.325
136860,136860,AVG,vs RHP,HitterId,0.295
150029,150029,AVG,vs RHP,HitterId,0.237
400121,400121,AVG,vs RHP,HitterId,0.242
400284,400284,AVG,vs RHP,HitterId,0.344
405395,405395,AVG,vs RHP,HitterId,0.406
407893,407893,AVG,vs RHP,HitterId,0.261
408047,408047,AVG,vs RHP,HitterId,0.269


In [160]:

df=pandas.DataFrame()


with open('combinations.txt', 'r') as combinations:

    foo=combinations.readline() #discard first line header row

    for line in combinations:
        
        args=line.split(',')
        
        args[2]=args[2].replace('\n','')

        df=df.append(create_combo_dataframe(raw_df, args[0], args[1], args[2]))
        
df.sort_values(['SubjectId', 'Stat', 'Split', 'Subject'], ascending=[True,True,True,True], inplace=True)

        
df.to_csv('test_full.csv', index=False)

UnboundLocalError: local variable 'df' referenced before assignment